In [ ]:
import pickle
import itertools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import csv

from utils import l2_to_l1

strategy_labels = ['Basic', 'Negative\n-in-area', 'Overlap', 'Cycle', 'Popularity']

## Main Analysis

In [ ]:
with open('data/maps.pkl', 'rb') as f:
    data = pickle.load(f)
    strategy_map = data['reviewer_to_strategy']

with open('data/results.pkl', 'rb') as f:
    results = pickle.load(f)
    
num_trials = 100
agg_results = {'success' : [], 'rank_simple' : [], 'rank_cluster' : [], 'rank_lowrank' : []}
for r in range(len(results['success_by_reviewer'])):
    strat = strategy_map[r]
    if strat < 0:
        continue
    total_success = int(results['success_by_reviewer'][r])
    total_failure = num_trials - total_success
    success_samples_flat = ([1] * total_success) + ([0] * total_failure)
    agg_results['success'] += [{'Success Rate' : x, 'rid' : r, 
                                'Strategy' : strategy_labels[strat]} 
                               for x in success_samples_flat]
    agg_results['rank_simple'] += [{'Average Detection Rank' : x, 'rid' : r, 
                                'Strategy' : strategy_labels[strat]} 
                               for x in results['rank_by_reviewer_simple'][r]]
    agg_results['rank_cluster'] += [{'Average Detection Rank' : x, 'rid' : r, 
                                'Strategy' : strategy_labels[strat]} 
                               for x in results['rank_by_reviewer_cluster'][r]]
    agg_results['rank_lowrank'] += [{'Average Detection Rank' : x, 'rid' : r, 
                                'Strategy' : strategy_labels[strat]} 
                               for x in results['rank_by_reviewer_lowrank'][r]]
    agg_results['rank_simple'] += [{'Average Detection Rank' : x, 'rid' : r, 
                                'Strategy' : 'Overall'} 
                               for x in results['rank_by_reviewer_simple'][r]]
    agg_results['rank_cluster'] += [{'Average Detection Rank' : x, 'rid' : r, 
                                'Strategy' : 'Overall'} 
                               for x in results['rank_by_reviewer_cluster'][r]]
    agg_results['rank_lowrank'] += [{'Average Detection Rank' : x, 'rid' : r, 
                                'Strategy' : 'Overall'} 
                               for x in results['rank_by_reviewer_lowrank'][r]]
    
dfs = {x : pd.DataFrame.from_records(y) for x, y in agg_results.items()}


sns.set(font_scale=1)
h = 3

In [ ]:
df = dfs['success']
g = sns.catplot(data=df, 
            x="Strategy", 
            y="Success Rate",
            errorbar='se', 
            kind="bar", 
            order=strategy_labels,
            color='r').set(ylim=(0, 1), 
                                ylabel='Manipulation Success Rate'
                               )
g.figure.set_figheight(h)
g.savefig('plots/success.pdf')

df = dfs['rank_simple']
g = sns.catplot(data=df, 
            x="Strategy", 
            y="Average Detection Rank",
            errorbar='se', 
            kind="bar", 
            order=strategy_labels+['Overall'],
            palette=(['b'] * len(strategy_labels) + ['lightblue'])
               ).set(ylim=(0, 28))
g.figure.set_figheight(h)
g.savefig('plots/simple.pdf')

df = dfs['rank_cluster']
g = sns.catplot(data=df, 
            x="Strategy", 
            y="Average Detection Rank",
            errorbar='se', 
            kind="bar", 
            order=strategy_labels+['Overall'],
            palette=(['b'] * len(strategy_labels) + ['lightblue'])
               ).set(ylim=(0, 28))
g.figure.set_figheight(h)
g.savefig('plots/cluster.pdf')

df = dfs['rank_lowrank']
g = sns.catplot(data=df, 
            x="Strategy", 
            y="Average Detection Rank",
            errorbar='se', 
            kind="bar", 
            order=strategy_labels+['Overall'],
            palette=(['b'] * len(strategy_labels) + ['lightblue'])
               ).set(ylim=(0, 28))
g.figure.set_figheight(h)
g.savefig('plots/lowrank.pdf')

## Synthetic Analysis

In [ ]:
group_size_label = 'Group Size'
n_label = '# Reviewers'
ns = [100, 500, 1000, 5000]
strats = [0, 1, 2, 3]
group_sizes = [2, 3, 4]

num_trials = 100
agg_results = {'success' : [], 'rank_simple' : [], 'rank_cluster' : [], 'rank_lowrank' : []}
for n, strategy, group_size in itertools.product(ns, strats, group_sizes):
    with open(f'data/synth_results/n_{n}__groupsize_{group_size}__strategy_{strategy}.pkl', 'rb') as f:
        results = pickle.load(f)
    total_success = sum(results['success'])
    total_failure = (group_size * len(results['success'])) - total_success
    success_samples_flat = ([1] * total_success) + ([0] * total_failure)
    if group_size == 1:
        print(n, strategy, total_success)
    assert len(success_samples_flat) == (group_size * num_trials)
    agg_results['success'] += [{'Success Rate' : x, group_size_label : group_size, 
                                n_label : n, 'Strategy' : strategy_labels[strategy]} 
                               for x in success_samples_flat]
    
    rank_simple_flat = [x for l in results['rank_simple'] for x in l]
    rank_cluster_flat = [x for l in results['rank_cluster'] for x in l]
    rank_lowrank_flat = [x for l in results['rank_lowrank'] for x in l]
    assert len(rank_simple_flat) == (group_size * num_trials)
    assert len(rank_cluster_flat) == (group_size * num_trials)
    assert len(rank_lowrank_flat) == (group_size * num_trials)
    agg_results['rank_simple'] += [{'Average Detection Rank' : x/n, group_size_label : group_size, 
                                n_label : n, 'Strategy' : strategy_labels[strategy]} 
                               for x in rank_simple_flat]
    agg_results['rank_cluster'] += [{'Average Detection Rank' : x/n, group_size_label : group_size, 
                                n_label : n, 'Strategy' : strategy_labels[strategy]} 
                               for x in rank_cluster_flat]
    agg_results['rank_lowrank'] += [{'Average Detection Rank' : x/n, group_size_label : group_size, 
                                n_label : n, 'Strategy' : strategy_labels[strategy]} 
                               for x in rank_lowrank_flat]

dfs = {x : pd.DataFrame.from_records(y) for x, y in agg_results.items()}

sns.set(font_scale=1.1)
h = 3

In [ ]:
success_df = dfs['success']
g = sns.catplot(data=success_df[success_df[n_label] == 5000], 
            x="Strategy", 
            y="Success Rate",
            errorbar='se', 
            hue=group_size_label, 
            kind="bar", 
            palette='Reds').set(ylim=(0, 1), 
                                ylabel='Manipulation Success Rate'
                               )
g.figure.set_figheight(h)
g.savefig('plots/synth_success_gs.pdf')

g = sns.catplot(data=success_df[success_df[group_size_label] == 4], 
            x="Strategy", 
            y="Success Rate",
            errorbar='se', 
            hue=n_label, 
            kind="bar", 
            palette='Reds').set(ylim=(0, 1), ylabel='Manipulation Success Rate')
g.figure.set_figheight(h)
g.savefig('plots/synth_success_n.pdf')

In [ ]:
df = dfs['rank_simple']
g = sns.catplot(data=df[df[n_label] == 5000], 
            x="Strategy", 
            y="Average Detection Rank",
            errorbar='se', 
            hue=group_size_label, 
            kind="bar", 
            palette='Blues').set(ylim=(0, 1), ylabel='Average Detection\n Rank (Normalized)')
g.figure.set_figheight(h)
g.savefig('plots/synth_simple_gs.pdf')

g = sns.catplot(data=df[df[group_size_label] == 4], 
            x="Strategy", 
            y="Average Detection Rank",
            errorbar='se', 
            hue=n_label, 
            kind="bar", 
            palette='Blues').set(ylim=(0, 1), ylabel='Average Detection\n Rank (Normalized)')
g.figure.set_figheight(h)
g.savefig('plots/synth_simple_n.pdf')

In [ ]:
df = dfs['rank_cluster']
g = sns.catplot(data=df[df[n_label] == 5000], 
            x="Strategy", 
            y="Average Detection Rank",
            errorbar='se', 
            hue=group_size_label, 
            kind="bar", 
            palette='Blues').set(ylim=(0, 1), ylabel='Average Detection\n Rank (Normalized)')
g.figure.set_figheight(h)
g.savefig('plots/synth_cluster_gs.pdf')

g = sns.catplot(data=df[df[group_size_label] == 4], 
            x="Strategy", 
            y="Average Detection Rank",
            errorbar='se', 
            hue=n_label, 
            kind="bar", 
            palette='Blues').set(ylim=(0, 1), ylabel='Average Detection\n Rank (Normalized)')
g.figure.set_figheight(h)
g.savefig('plots/synth_cluster_n.pdf')

In [ ]:
df = dfs['rank_lowrank']
g = sns.catplot(data=df[df[n_label] == 5000], 
            x="Strategy", 
            y="Average Detection Rank",
            errorbar='se', 
            hue=group_size_label, 
            kind="bar", 
            palette='Blues').set(ylim=(0, 1), ylabel='Average Detection\n Rank (Normalized)')
g.figure.set_figheight(h)
g.savefig('plots/synth_lowrank_gs.pdf')

g = sns.catplot(data=df[df[group_size_label] == 4], 
            x="Strategy", 
            y="Average Detection Rank",
            errorbar='se', 
            hue=n_label, 
            kind="bar", 
            palette='Blues').set(ylim=(0, 1), ylabel='Average Detection\n Rank (Normalized)')
g.figure.set_figheight(h)
g.savefig('plots/synth_lowrank_n.pdf')

## Bid Distributions

In [ ]:
sns.set(font_scale=1.4)
h = 3

data = np.load('data/Biddings.npz')
HB = data['HB']
MB = data['MB']
records = []
for bid, label in zip([-1, 0, 1], ['Negative', 'Neutral', 'Positive']):
    h_counts = np.sum(HB == bid, axis=1)
    m_counts = np.sum(MB == bid, axis=1)
    for hc, mc in zip(h_counts, m_counts):
        records += [{'count' : hc, 'bid_value' : bid, 'Type' : 'Honest'},
                      {'count' : mc, 'bid_value' : bid, 'Type' : 'Malicious'}]
df = pd.DataFrame.from_records(records)

In [ ]:
g = sns.histplot(data=df[df['bid_value'] == -1], 
                 x='count', 
                 hue='Type', 
                 palette=('grey', 'red'), 
                 multiple="dodge",shrink=0.8,
                 discrete=True)
g.set(ylim=(0, 15), xlabel="# Negative Bids", ylabel='# Participants')
g.set_xticks(np.arange(0, 26, 3))
g.figure.set_figheight(h)
g.figure.savefig('plots/dist_neg.pdf', bbox_inches = "tight")

In [ ]:
g = sns.histplot(data=df[df['bid_value'] == 1], 
                 x='count', 
                 hue='Type', 
                 palette=('grey', 'red'), 
                 multiple="dodge", shrink=0.8,
                 discrete=True)
g.set(ylim=(0, 15), xlabel="# Positive Bids", ylabel='# Participants')
g.set_xticks(np.arange(0, 15, 3))
g.figure.set_figheight(h)
g.figure.savefig('plots/dist_pos.pdf', bbox_inches = "tight")

## Other Info

In [ ]:
with open('data/maps.pkl', 'rb') as f:
    data = pickle.load(f)
    strategy_map = data['reviewer_to_strategy']
    strat_descs = data['strategy_descriptions']
    author_map_group = data['author_map_group']
    target_map = data['target_map']
    author_map = data['author_map']
    
data = np.load('data/Biddings.npz')
MB = data['MB']
SA = data['SA']
    
with open('data/strat_analysis.csv', 'w') as f:
    f.write('Strategy\tDescription\tTarget Bids\t#Similar/Positive\t#Similar/Negative\t#Similar/Neutral\t#Nonsimilar/Positive\t#Nonsimilar/Negative\t#Nonsimilar/Neutral\n')
    for rid in range(31):
        strat = strategy_map[rid]
        desc = strat_descs[rid].replace('\t', ' ').replace('\n', ' ')
        if rid in target_map:
            target = [target_map[rid]]
        else:
            target = author_map_group[rid].copy()
            authored = author_map[rid]
            target.remove(authored)
        SA_row = SA[rid, :]
        B_row = MB[rid, :]
        papers_sim = [x[0] for x in np.argwhere(SA_row > 0) if x[0] not in target] 
        papers_non = [x[0] for x in np.argwhere(SA_row == 0) if x[0] not in target]
        target_bids = B_row[target].tolist()
        pos = B_row == 1
        neg = B_row == -1
        neut= B_row == 0
        npos = [np.sum(pos[papers_sim]), np.sum(pos[papers_non])]
        nneg = [np.sum(neg[papers_sim]), np.sum(neg[papers_non])]
        nneut = [np.sum(neut[papers_sim]), np.sum(neut[papers_non])]
        f.write(f'{strat}\t{desc}\t{target_bids}\t{npos[0]}\t{nneg[0]}\t{nneut[0]}\t{npos[1]}\t{nneg[1]}\t{nneut[0]}\n')

In [ ]:
with open('data/maps.pkl', 'rb') as f:
    data = pickle.load(f)
    r2s = data['reviewer_to_sas']
    p2s = data['paper_to_sas']
    
count = np.zeros(7)
for i, s in r2s.items():
    q = { l2_to_l1(j) for j in s}
    for j in q:
        count[j] += 1
print("Reviewer areas:", count)
count = np.zeros(7)
for i, s in p2s.items():
    q = l2_to_l1(s) 
    count[q] += 1
print("Paper areas:", count)